### Developer: Mani kanta
### Dependence : S3
### Aim: Data Preprocessing for generating Hotel Embeddings from User Clickstream Sessions on Listing page

# Importing Libraries

In [2]:
import pandas as pd
import torch
from tqdm import tqdm
import numpy as np

In [3]:
hotel_data = pd.read_csv("Hotel.csv")

In [4]:
hotel_data

,Unnamed: 0,hotel_id,city_id,city_name,country_id
0,0,94319,678.0,Nha-Trang,217
1,1,13195,4.0,Bangalore,1
2,2,89191,4.0,Bangalore,1
3,3,86999,421.0,Karimnagar,1
4,4,87204,677.0,Krabi,8
...,...,...,...,...,...
81412,81412,62801,1.0,Gurgaon,1
81413,81413,97059,14.0,Kolkata,1
81414,81414,110181,1.0,Gurgaon,1
81415,81415,5298,5.0,Mumbai,1


In [7]:
#Mapping hotel_id -> city_id,country_id 
#Helps in creating negative samples

hotel_to_city_id = {}
for idx,item in enumerate(hotel_data['hotel_id']):
    hotel_to_city_id[item] = [hotel_data['city_id'][idx],hotel_data['country_id'][idx]]
    

In [9]:
hotel_to_city_id[94319]

[678.0, 217]

In [10]:
all_hotels = list(hotel_data["hotel_id"])

In [11]:
all_hotels

[94319,
 13195,
 89191,
 86999,
 87204,
 4100,
 59696,
 51925,
 91694,
 82810,
 77364,
 54587,
 13586,
 61433,
 112901,
 44715,
 100984,
 107909,
 76652,
 3987,
 109078,
 57473,
 89498,
 15621,
 43033,
 84197,
 90434,
 84446,
 89046,
 104491,
 49603,
 101763,
 84704,
 102296,
 85919,
 17771,
 100110,
 13254,
 3287,
 665,
 13843,
 86349,
 6604,
 1144,
 77373,
 81183,
 4914,
 106766,
 14453,
 4084,
 7405,
 110437,
 7455,
 44186,
 1388,
 98032,
 41425,
 54008,
 27291,
 37056,
 107503,
 17227,
 50450,
 82128,
 103584,
 60705,
 62760,
 9630,
 76374,
 62137,
 12496,
 38814,
 19689,
 102988,
 47400,
 101419,
 17553,
 84308,
 98191,
 15041,
 85878,
 105231,
 16760,
 105608,
 37372,
 86118,
 38045,
 111011,
 43487,
 6019,
 49936,
 53514,
 6347,
 80122,
 4445,
 75122,
 43578,
 76362,
 89133,
 16106,
 58365,
 51350,
 101924,
 87294,
 8598,
 73732,
 12927,
 106761,
 104207,
 101382,
 55215,
 19334,
 45386,
 84282,
 40343,
 53822,
 100567,
 112683,
 50687,
 110822,
 958,
 40754,
 5859,
 102736,
 11

In [16]:
df_para = pd.read_parquet('part-00000-34fc8918-9837-47bb-9d5e-c1babbd238a5-c000.snappy.parquet', engine='fastparquet')

In [15]:
#!pip install fastparquet

     |████████████████████████████████| 1.2 MB 21.2 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 75.0 MB/s eta 0:00:01
  Created wheel for thrift: filename=thrift-0.15.0-cp36-cp36m-linux_x86_64.whl size=181323 sha256=1a23e0b5e32541ee70d2abfbf755d8fd4a108286d8c7a402bb6c6e1d32332e5d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2a/91/a0/aea72c790bd97f9c2f31ac5ca0b20b25992010a31e4357f50b
Successfully built thrift


In [17]:
df_para

,requestid,sorted_hotels,booked_hotel_id
0,00c7344b-c1c6-47ad-86c9-a4fafa6870ad,"[91354, 105845, 105312, 88931]",None
1,01e02dae-31f1-4741-be9e-f337a726b368,"[55759, 55759, 55759, 55759, 55759]",None
2,02237619-9013-46d3-9348-43bd03ed01b6||google_r...,"[106631, 106631, 106631]",None
3,027ab011-a43f-46e8-bd2b-ed306f6886e8||google_r...,"[48059, 48059, 14664, 14664]",14664
4,027ae728-d700-4136-8d7a-2681e5679b78||oyo-loca...,"[50419, 50419, 50419]",None
...,...,...,...
111709,fd03564f-dd9d-4ae6-8bce-557b2cc5a963,"[36835, 36835, 36835]",None
111710,fdcb6737-f2e3-47ec-9178-39adb15eb0a1,"[108819, 106942, 106942, 106942, 109046]",None
111711,fe31b2cd-6933-4234-86ce-373ab7a9cf2e,"[38259, 17061, 17061, 17061, 105939, 108130, 1...",None
111712,fe9a66b5-acd0-4174-9f55-d50bf458d731||google_r...,"[94978, 102627, 40877]",None


In [18]:
temp_para = df_para.copy()

In [19]:
#removing hotel_ids with consecutive clicks [15411, 15411, 15411, 15411,1234] -> [15411,1234]

def remove_duplicates(hotel_ids_list):
    res = [i[0] for i in groupby(hotel_ids_list)]
    return res

In [21]:
from itertools import groupby

temp_para["sorted_hotels"] = temp_para["sorted_hotels"].apply(remove_duplicates)

In [22]:
temp_para

,requestid,sorted_hotels,booked_hotel_id
0,00c7344b-c1c6-47ad-86c9-a4fafa6870ad,"[91354, 105845, 105312, 88931]",None
1,01e02dae-31f1-4741-be9e-f337a726b368,[55759],None
2,02237619-9013-46d3-9348-43bd03ed01b6||google_r...,[106631],None
3,027ab011-a43f-46e8-bd2b-ed306f6886e8||google_r...,"[48059, 14664]",14664
4,027ae728-d700-4136-8d7a-2681e5679b78||oyo-loca...,[50419],None
...,...,...,...
111709,fd03564f-dd9d-4ae6-8bce-557b2cc5a963,[36835],None
111710,fdcb6737-f2e3-47ec-9178-39adb15eb0a1,"[108819, 106942, 109046]",None
111711,fe31b2cd-6933-4234-86ce-373ab7a9cf2e,"[38259, 17061, 105939, 108130]",None
111712,fe9a66b5-acd0-4174-9f55-d50bf458d731||google_r...,"[94978, 102627, 40877]",None


In [23]:
temp_para.dtypes

requestid          object
sorted_hotels      object
booked_hotel_id    object
dtype: object

In [25]:
#sessions ending with Bookings

temp_para[temp_para["booked_hotel_id"].notnull()] 

,requestid,sorted_hotels,booked_hotel_id
3,027ab011-a43f-46e8-bd2b-ed306f6886e8||google_r...,"[48059, 14664]",14664
31,0f4dffa4-713c-4cab-8145-2c39e598684d||oyo-loca...,"[39850, 1600]",1600
57,1b56af38-c76e-4526-9ab1-45f2f5404284||google_r...,"[79297, 104964, 108123, 79297]",79297
58,1b879e0c-78df-40b9-a5fc-10f7be8c92ef||A8BE453E...,[107533],107533
71,24a584e3-e661-4667-a1a3-1c76284ecf04||google_r...,"[18076, 106821, 107869, 548, 105854, 548, 18076]",18076
...,...,...,...
111616,d2eaba9b-2231-4211-a1a1-0af4f60b94ef||10166412,[95103],95103
111627,da171755-b3db-4bab-a11d-bbd11d9e0acb||oyo-loca...,"[105749, 108240]",108240
111630,db996ec4-7dec-4ca3-aed9-1fe1969a90f4,"[16580, 102955, 109046, 16580]",16580
111660,e5f7c7d6-0737-4ee1-a0fd-cf3b535b91ff||google_r...,[48783],48783


In [26]:
#sessions with at least 3 different hotel clicks 

data = temp_para[temp_para["sorted_hotels"].str.len() > 3]

In [37]:
data = data.reset_index()

In [42]:
data.columns

Index(['index', 'requestid', 'sorted_hotels', 'booked_hotel_id'], dtype='object')

In [44]:
data.drop(['index'],axis =1,inplace= True)

In [45]:
data

,requestid,sorted_hotels,booked_hotel_id
0,00c7344b-c1c6-47ad-86c9-a4fafa6870ad,"[91354, 105845, 105312, 88931]",None
1,074eaad5-69a1-490d-96a9-aea48b990ec6||google_r...,"[109186, 105290, 59871, 76893]",None
2,0ef431ca-74ea-4d4e-a945-178f6f8226a0,"[106168, 74109, 8149, 108713, 8148, 109026, 823]",None
3,0ff94e31-45db-46ac-bb2c-fd232dd87397||oyo-loca...,"[104204, 74887, 73644, 108123, 79297, 95959]",None
4,10a70791-d6b3-4d8b-8752-0203853eae0d||google_r...,"[106926, 55396, 102164, 106926]",None
...,...,...,...
25424,f6e16a85-709c-4d03-b8fb-d9f37eb0109c||google_r...,"[104671, 9517, 56493, 75018]",None
25425,f844bba4-cf67-40c9-a792-4fe457ec91d5,"[105382, 108402, 104386, 105462]",None
25426,f8c753ec-ac82-46f4-81a3-b0a4ab1f614b||google_r...,"[41671, 87332, 13825, 110550]",None
25427,fbf2d939-1ee7-4707-b1c4-b04663007cb3,"[106839, 107173, 104614, 110781]",None


In [47]:
click_streams = []

for idx,hotel_ids in enumerate(data["sorted_hotels"]):
#     print(idx)
    if data["booked_hotel_id"][idx] != None:
        click_streams.append([hotel_ids[:-1],data["booked_hotel_id"][idx] ])
    else:
        click_streams.append([hotel_ids,data["booked_hotel_id"][idx] ])


In [48]:
click_streams

[[['91354', '105845', '105312', '88931'], None],
 [['109186', '105290', '59871', '76893'], None],
 [['106168', '74109', '8149', '108713', '8148', '109026', '823'], None],
 [['104204', '74887', '73644', '108123', '79297', '95959'], None],
 [['106926', '55396', '102164', '106926'], None],
 [['93376', '109810', '62261', '86623', '109810'], None],
 [['110681',
   '78734',
   '84710',
   '110156',
   '109873',
   '110494',
   '108587',
   '109526',
   '109344',
   '74908',
   '42137',
   '1730',
   '108780',
   '99796',
   '107895',
   '55008',
   '109612',
   '109447',
   '109837',
   '110086'],
  None],
 [['108308', '78759', '95769', '48744'], None],
 [['76239', '109204', '35299', '76433'], None],
 [['92102',
   '109002',
   '61770',
   '109002',
   '102380',
   '92102',
   '82199',
   '85308',
   '82199'],
  None],
 [['109476', '108614', '57351', '110682', '107829'], None],
 [['79573', '74578', '79573', '109821'], None],
 [['79297', '104964', '108123'], '79297'],
 [['107436', '110299', '

In [49]:
count = 0
for item in click_streams:
    if item[1] != None:
        print(item)
        count = count+1


[['79297', '104964', '108123'], '79297']
[['18076', '106821', '107869', '548', '105854', '548'], '18076']
[['49739', '110563', '98690'], '105396']
[['37897', '3545', '37897'], '3545']
[['110055', '109755', '109757', '109521', '57207', '90959', '103417', '104651'], '107437']
[['18284', '108009', '45743', '18672'], '46381']
[['110353', '94615', '94646', '106629'], '110353']
[['109612', '12515', '109916', '12515', '109612', '17185', '109612', '110699'], '107938']
[['108537', '104974', '108537', '109052'], '104974']
[['81963', '10593', '2600', '81963', '74747', '104671', '2600', '10237', '42742', '46858', '81963', '2600', '10327'], '2600']
[['105612', '109489', '107216', '101841', '105296', '107216', '98195', '105296', '90714'], '107216']
[['107044', '104507', '94574'], '104507']
[['48783', '46457', '88430'], '78399']
[['107134', '109245', '109244'], '109245']
[['106167', '107774', '106167', '105262', '98811', '110502', '85074'], '105262']
[['106113', '105464', '106113'], '105464']
[['1077

In [50]:
count

1866

In [51]:
def gather_word_freqs(hotels_list): #here split_text is sent_list

        vocab = {}
        ix_to_word = {}
        word_to_ix = {}
        total = 0.0

        print('building vocab...')
        for word in hotels_list: #for every word in the word list(split_text), which might occur multiple times
            if word not in vocab:
                vocab[word] = 0
                ix_to_word[len(word_to_ix)] = word
                word_to_ix[word] = len(word_to_ix)
            vocab[word] += 1.0 #count of the word stored in a dict
            total += 1.0 #total number of words in the word_list(split_text)

        return vocab, word_to_ix, ix_to_word

In [52]:
vocab, word_to_ix, ix_to_word = gather_word_freqs(all_hotels)

building vocab...


# Making Traning Data by taking window size = 2

In [53]:
def gather_training_data(split_text, word_to_ix, context_size):        
        training_data = []
        all_vocab_indices = list(range(len(word_to_ix)))
        
        #for each sentence
        print('preparing training data (x, y)...')
        for sentence in tqdm(split_text):
            if sentence[1] != None:
                sentence[0].append(sentence[1])
                
            indices = [word_to_ix[int(word)] for word in sentence[0]]
            
            #for each word treated as center word
            for center_word_pos in range(len(indices)):
                
                #for each window  position
                temp_context = []
                center_word_idx  = indices[center_word_pos]
                for w in range(-context_size, context_size+1):                
                    context_word_pos = center_word_pos + w

                    #make sure we dont jump out of the sentence
                    if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                        continue
                    
                    context_word_idx = indices[context_word_pos]
                    
                    if center_word_idx == context_word_idx: # same words might be present in the close vicinity of each other. we want to avoid such cases
                        continue
                        
                    temp_context.append(context_word_idx)
                    
                    
                if sentence[1] != None :
                    temp_context.append(word_to_ix[int(sentence[1])])
                    
                training_data.append([center_word_idx, temp_context])
        
        return training_data

In [54]:
train_data = gather_training_data(click_streams, word_to_ix, 2 )

preparing training data (x, y)...


100%|██████████| 25429/25429 [00:00<00:00, 53011.81it/s]


In [55]:
train_data

[[55544, [59195, 32321]],
 [59195, [55544, 32321, 59346]],
 [32321, [55544, 59195, 59346]],
 [59346, [59195, 32321]],
 [58304, [43818, 78842]],
 [43818, [58304, 78842, 5794]],
 [78842, [58304, 43818, 5794]],
 [5794, [43818, 78842]],
 [55400, [29667, 22159]],
 [29667, [55400, 22159, 28081]],
 [22159, [55400, 29667, 28081, 23032]],
 [28081, [29667, 22159, 23032, 50683]],
 [23032, [22159, 28081, 50683, 647]],
 [50683, [28081, 23032, 647]],
 [647, [23032, 50683]],
 [44709, [73223, 51888]],
 [73223, [44709, 51888, 61412]],
 [51888, [44709, 73223, 61412, 45698]],
 [61412, [73223, 51888, 45698, 10448]],
 [45698, [51888, 61412, 10448]],
 [10448, [61412, 45698]],
 [17153, [66779, 77503]],
 [66779, [17153, 77503, 17153]],
 [77503, [17153, 66779, 17153]],
 [17153, [66779, 77503]],
 [24776, [10941, 30615]],
 [10941, [24776, 30615, 60829]],
 [30615, [24776, 10941, 60829, 10941]],
 [60829, [10941, 30615, 10941]],
 [10941, [30615, 60829]],
 [52762, [45243, 4943]],
 [45243, [52762, 4943, 50938]],
 [49

In [56]:
positive_data = train_data

# Making Negative Samples

In [57]:
city_neg = []
country_neg = []
for item in tqdm(train_data):
    center_id = item[0]
    city_id, country_id = hotel_to_city_id[int(ix_to_word[center_id])]
    city_hotels = d[country_id][city_id]
    
    within_city = []
    within_country = []
    for i in range(2):
        within_city.extend([word_to_ix[id_] for id_ in list(np.random.choice(city_hotels, 2))])
        within_country.extend( [ word_to_ix[id_] for id_ in list(np.random.choice(all_hotels, 2)) ])
    
    city_neg.append(within_city)
    country_neg.append(within_country)

100%|██████████| 143509/143509 [26:15<00:00, 91.07it/s]


In [65]:
train_data_numpy = np.array(train_data)
within_city = np.array(city_neg)
within_country = np.array(country_neg)

ValueError: setting an array element with a sequence.

In [59]:
with open('train_data.npy', 'wb') as f:
    np.save(f, train_data_numpy)

In [60]:
with open('within_city.npy', 'wb') as f:
    np.save(f, within_city)

In [61]:
with open('within_country.npy', 'wb') as f:
    np.save(f, within_country)

In [62]:
len(train_data_numpy)

143509

In [66]:
center_id = 

AttributeError: 'list' object has no attribute 'shape'

In [64]:
train_data_numpy

array([[55544, list([59195, 32321])],
       [59195, list([55544, 32321, 59346])],
       [32321, list([55544, 59195, 59346])],
       ...,
       [13498, list([22858, 65386, 37983])],
       [65386, list([22858, 13498, 37983])],
       [37983, list([13498, 65386])]], dtype=object)

In [71]:
center_id = train_data_numpy[:,0] 

In [77]:
center_id = np.array(center_id,dtype = int)

In [78]:
center_id

array([55544, 59195, 32321, ..., 13498, 65386, 37983])

In [80]:
train_data_numpy[:,1] 

array([list([59195, 32321]), list([55544, 32321, 59346]),
       list([55544, 59195, 59346]), ..., list([22858, 65386, 37983]),
       list([22858, 13498, 37983]), list([13498, 65386])], dtype=object)

In [107]:
positive_context = np.array(train_data[:,1] ,dtype = int)

TypeError: list indices must be integers or slices, not tuple

In [116]:
positive_context = np.array([[ele for ele in item[1]] for item in train_data[:10]])

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


In [125]:
a = list(positive_context[:2])

In [136]:
positive_context[:2].shape

(2,)

In [137]:
a

[[59195, 32321], [55544, 32321, 59346]]

In [145]:
[print((np.array(item[1]))) for item in train_data[:10]]

[59195 32321]
[55544 32321 59346]
[55544 59195 59346]
[59195 32321]
[43818 78842]
[58304 78842  5794]
[58304 43818  5794]
[43818 78842]
[29667 22159]
[55400 22159 28081]


[None, None, None, None, None, None, None, None, None, None]

In [ ]:
center_id , []

In [130]:
type(a[0])

list

In [122]:
positive_context = np.asarray(positive_context[:10], dtype=float)

ValueError: setting an array element with a sequence.

In [120]:
positive_context.

array([list([59195, 32321]), list([55544, 32321, 59346]),
       list([55544, 59195, 59346]), list([59195, 32321]),
       list([43818, 78842]), list([58304, 78842, 5794]),
       list([58304, 43818, 5794]), list([43818, 78842]),
       list([29667, 22159]), list([55400, 22159, 28081])], dtype=object)

In [103]:
one_list = train_data[1][1]

In [105]:
type(one_list)

list

In [106]:
np.array(one_list)

array([55544, 32321, 59346])

In [108]:
train_data[1]

[59195, [55544, 32321, 59346]]

In [99]:
positive_context[:2]

array([list([59195, 32321]), list([55544, 32321, 59346])], dtype=object)

In [1]:
1013795/143509

7.064330460110516

In [2]:
495360/143509

3.451769575427325